En el pair programming de hoy usaremos el set de datos que guardastéis en el pair programming de normalización y estandarización.


In [48]:
# Tratamiento de datos
# -----------------------------------------------------------------------
import numpy as np
import pandas as pd
import random 

# Gráficos
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# Estadísticos
# ------------------------------------------------------------------------------
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.multivariate.manova import MANOVA
from sklearn.preprocessing import StandardScaler

plt.rcParams["figure.figsize"] = (10,8) 

In [49]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [50]:
df = pd.read_csv("data/df_norm_estand.csv", index_col = 0)
df.sample(5)

,age,sex,bmi,children,smoker,region,charges,charges_Sklearn,age_robust,bmi_robust,children_robust
397,21,male,31.020,0,no,southeast,16586.49771,0.143596,0.04,-0.016833,-0.5
1164,41,female,28.310,1,no,northwest,7153.55390,0.047643,-0.80,-0.497506,0.0
713,20,male,40.470,0,no,northeast,1984.45330,0.094279,-0.36,1.806110,0.5
1308,25,female,30.200,0,yes,southwest,33900.65300,NaN,NaN,NaN,NaN
367,42,female,24.985,2,no,northwest,8017.06115,0.030375,-0.80,0.675187,-0.5


In [51]:
df.isnull().sum()

age                  0
sex                  0
bmi                  0
children             0
smoker               0
region               0
charges              0
charges_Sklearn    125
age_robust         125
bmi_robust         125
children_robust    125
dtype: int64

### Empezamos con el método OLS

In [52]:
lm = ols('charges_Sklearn ~ age + sex + bmi + children + smoker + region',data = df).fit()

In [53]:
sm.stats.anova_lm(lm)

,df,sum_sq,mean_sq,F,PR(>F)
sex,1.0,0.004750,0.004750,0.100086,0.751789
smoker,1.0,0.023596,0.023596,0.497185,0.480894
region,3.0,0.138879,0.046293,0.975417,0.403554
age,1.0,0.026743,0.026743,0.563495,0.453021
bmi,1.0,0.017450,0.017450,0.367675,0.544404
children,1.0,0.133710,0.133710,2.817334,0.093545
Residual,1064.0,50.497179,0.047460,NaN,NaN


## Método summary 

In [54]:
lm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        charges_Sklearn   R-squared:                       0.007
Model:                            OLS   Adj. R-squared:                 -0.001
Method:                 Least Squares   F-statistic:                    0.9090
Date:                Mon, 23 Jan 2023   Prob (F-statistic):              0.508
Time:                        12:22:41   Log-Likelihood:                 117.18
No. Observations:                1073   AIC:                            -216.4
Df Residuals:                    1064   BIC:                            -171.6
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
=======================================================================================
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept               0.3105      0.041      7.495      0.000       0.229       0.392
sex[T.male]            -0.0044      0.013     -0.330      0.741      -0.031       0.022
smoker[T.yes]           0.0122      0.022      0.552      0.581      -0.031       0.055
region[T.northwest]    -0.0188      0.019     -1.000      0.318      -0.056       0.018
region[T.southeast]    -0.0069      0.019     -0.355      0.722      -0.045       0.031
region[T.southwest]     0.0137      0.019      0.717      0.473      -0.024       0.051
age                    -0.0003      0.000     -0.623      0.534      -0.001       0.001
bmi                    -0.0007      0.001     -0.604      0.546      -0.003       0.002
children               -0.0092      0.006     -1.678      0.094      -0.020       0.002
==============================================================================
Omnibus:                      197.454   Durbin-Watson:                   2.036
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              320.743
Skew:                           1.206   Prob(JB):                     2.25e-70
Kurtosis:                       4.163   Cond. No.                         328.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## Vamos a ver con efectos aditivos 

In [55]:
lm = ols('charges_Sklearn ~ age_robust * bmi_robust * children_robust * smoker * sex * region',data = df).fit()

In [56]:
sm.stats.anova_lm(lm)

,df,sum_sq,mean_sq,F,PR(>F)
smoker,1.0,0.023199,0.023199,0.607195,0.436041
sex,1.0,0.005147,0.005147,0.134718,0.713672
region,3.0,0.138879,0.046293,1.211634,0.304308
smoker:sex,1.0,0.000854,0.000854,0.022360,0.881165
smoker:region,3.0,0.091986,0.030662,0.802517,0.492563
...,...,...,...,...,...
age_robust:bmi_robust:children_robust:smoker:sex,1.0,0.136328,0.136328,3.568136,0.059204
age_robust:bmi_robust:children_robust:smoker:region,3.0,0.058872,0.019624,0.513618,0.672975
age_robust:bmi_robust:children_robust:sex:region,3.0,0.188243,0.062748,1.642303,0.177999
age_robust:bmi_robust:children_robust:smoker:sex:region,3.0,0.126744,0.042248,1.105762,0.345806


In [57]:
lm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        charges_Sklearn   R-squared:                       0.290
Model:                            OLS   Adj. R-squared:                  0.194
Method:                 Least Squares   F-statistic:                     3.037
Date:                Mon, 23 Jan 2023   Prob (F-statistic):           6.58e-22
Time:                        12:22:41   Log-Likelihood:                 297.16
No. Observations:                1073   AIC:                            -338.3
Df Residuals:                     945   BIC:                             298.9
Df Model:                         127                                         
Covariance Type:            nonrobust                                         
=======================================================================================================================================================
                                                                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                                               0.2491      0.019     13.256      0.000       0.212       0.286
smoker[T.yes]                                                                           0.0779      0.058      1.335      0.182      -0.037       0.192
sex[T.male]                                                                             0.0314      0.027      1.158      0.247      -0.022       0.085
region[T.northwest]                                                                     0.0417      0.026      1.591      0.112      -0.010       0.093
region[T.southeast]                                                                     0.0127      0.026      0.488      0.626      -0.038       0.064
region[T.southwest]                                                                     0.0223      0.027      0.841      0.401      -0.030       0.074
smoker[T.yes]:sex[T.male]                                                              -0.0706      0.088     -0.806      0.421      -0.243       0.101
smoker[T.yes]:region[T.northwest]                                                      -0.1837      0.100     -1.845      0.065      -0.379       0.012
smoker[T.yes]:region[T.southeast]                                                      -0.0874      0.139     -0.629      0.530      -0.360       0.185
smoker[T.yes]:region[T.southwest]                                                      -0.4924      0.305     -1.613      0.107      -1.092       0.107
sex[T.male]:region[T.northwest]                                                        -0.0949      0.038     -2.521      0.012      -0.169      -0.021
sex[T.male]:region[T.southeast]                                                        -0.0455      0.038     -1.213      0.226      -0.119       0.028
sex[T.male]:region[T.southwest]                                                        -0.0364      0.038     -0.952      0.341      -0.111       0.039
smoker[T.yes]:sex[T.male]:region[T.northwest]                                           0.1770      0.135      1.313      0.190      -0.088       0.442
smoker[T.yes]:sex[T.male]:region[T.southeast]                                           0.1576      0.170      0.927      0.354      -0.176       0.491
smoker[T.yes]:sex[T.male]:region[T.southwest]                                           0.4368      0.319      1.368      0.172      -0.190       1.063
age_robust                                                                              0.1666      0.032      5.172      0.000       0.103       0.230
age_robust:smoker[T.yes]                   

## Vemos con efectos no aditivos pero con el Robust

In [58]:
df.columns

Index(['age', 'sex', 'bmi', 'children', 'smoker', 'region', 'charges',
       'charges_Sklearn', 'age_robust', 'bmi_robust', 'children_robust'],
      dtype='object')

In [59]:
lm = ols('charges_Sklearn ~ age_robust + bmi_robust + children_robust + smoker + sex + region',data = df).fit()

In [60]:
sm.stats.anova_lm(lm)

,df,sum_sq,mean_sq,F,PR(>F)
smoker,1.0,0.023199,0.023199,0.613509,4.336439e-01
sex,1.0,0.005147,0.005147,0.136119,7.122435e-01
region,3.0,0.138879,0.046293,1.224235,2.996054e-01
age_robust,1.0,9.507825,9.507825,251.437617,5.414387e-51
bmi_robust,1.0,0.716089,0.716089,18.937204,1.481004e-05
children_robust,1.0,0.217227,0.217227,5.744653,1.671066e-02
Residual,1064.0,40.233941,0.037814,NaN,NaN


In [61]:
lm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        charges_Sklearn   R-squared:                       0.209
Model:                            OLS   Adj. R-squared:                  0.203
Method:                 Least Squares   F-statistic:                     35.07
Date:                Mon, 23 Jan 2023   Prob (F-statistic):           2.01e-49
Time:                        12:22:43   Log-Likelihood:                 239.08
No. Observations:                1073   AIC:                            -460.2
Df Residuals:                    1064   BIC:                            -415.4
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
=======================================================================================
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept               0.2721      0.013     20.168      0.000       0.246       0.299
smoker[T.yes]           0.0189      0.019      0.997      0.319      -0.018       0.056
sex[T.male]            -0.0161      0.012     -1.356      0.176      -0.040       0.007
region[T.northwest]    -0.0124      0.017     -0.736      0.462      -0.045       0.021
region[T.southeast]    -0.0128      0.017     -0.764      0.445      -0.046       0.020
region[T.southwest]     0.0143      0.017      0.841      0.400      -0.019       0.048
age_robust              0.1724      0.011     16.145      0.000       0.151       0.193
bmi_robust             -0.0350      0.008     -4.353      0.000      -0.051      -0.019
children_robust         0.0236      0.010      2.397      0.017       0.004       0.043
==============================================================================
Omnibus:                      450.247   Durbin-Watson:                   2.033
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1583.693
Skew:                           2.094   Prob(JB):                         0.00
Kurtosis:                       7.229   Cond. No.                         5.45
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Intepretación

Con el coeficiente podemos ver que todas nuestras variables tienen una relación postiva, menos las de 'region', que es negativa.

El std error 

El r-squared es el porcentaje de la variacion de la variable respuesta. ES decir, nuestras variables predictoras explican el 40,2% de la variacion en los precios del seguro medico (charges). ES un valor bajo.